In [1]:
import pandas as pd
import numpy as np
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
'done'

'done'

In [2]:
df = pd.concat([train,test], axis = 0, sort = False)

# Factorize

In [3]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / df[col2]

def multiply_name(col1, col2): return col1 + '_times_' + col2
def multiply(df, col1, col2): return df[col1] * df[col2]

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col, dummy_na = True)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [4]:
flag_other = [col for col in train.columns.values if 'FLAG' in col and 'DOCUMENT' not in col]
df['CONTACT'] = df[['FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE','FLAG_EMAIL']].sum(axis = 1)

df.drop(['FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE'], axis = 1, inplace = True)

df['FLAG_OTHER'] = df[['FLAG_OWN_CAR',
 'FLAG_OWN_REALTY']].sum(axis = 1)
print(df.shape)

(356255, 119)


## Factorize & onehot

In [5]:
categorical_columns = [col for col in df.columns if df[col].dtype == 'object'] + ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

factor_columns = [col for col in categorical_columns if len(set(df[col])) == 2] + ['CODE_GENDER', 'WEEKDAY_APPR_PROCESS_START']

categorical_columns = [i for i in categorical_columns if i not in factor_columns]

for bin_feature in factor_columns:
    print(bin_feature)
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

print(len(categorical_columns),len(factor_columns))
for col in categorical_columns:
    print(col)
    print(set(df[col]))

NAME_CONTRACT_TYPE
FLAG_OWN_CAR
FLAG_OWN_REALTY
FLAG_OWN_CAR
FLAG_OWN_REALTY
CODE_GENDER
WEEKDAY_APPR_PROCESS_START
12 7
NAME_TYPE_SUITE
{nan, 'Family', 'Spouse, partner', 'Unaccompanied', 'Other_A', 'Other_B', 'Group of people', 'Children'}
NAME_INCOME_TYPE
{'Pensioner', 'State servant', 'Maternity leave', 'Student', 'Working', 'Unemployed', 'Commercial associate', 'Businessman'}
NAME_EDUCATION_TYPE
{'Academic degree', 'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Higher education'}
NAME_FAMILY_STATUS
{'Civil marriage', 'Married', 'Single / not married', 'Widow', 'Unknown', 'Separated'}
NAME_HOUSING_TYPE
{'Rented apartment', 'Co-op apartment', 'Municipal apartment', 'House / apartment', 'Office apartment', 'With parents'}
OCCUPATION_TYPE
{nan, 'Drivers', 'Sales staff', 'Medicine staff', 'Cooking staff', 'Private service staff', 'Cleaning staff', 'Waiters/barmen staff', 'Realty agents', 'Accountants', 'Secretaries', 'Security staff', 'Managers', 'High skill 

In [6]:
ignore_cat = ['ORGANIZATION_TYPE'] # 'NAME_HOUSING_TYPE','OCCUPATION_TYPE',
for col in categorical_columns:
    if col not in ignore_cat:
        tmp = pd.get_dummies(df[col], prefix= 'one_hot'+col, dummy_na = True)
        print(tmp.shape)
        print(col)
        df = pd.concat([df, tmp], axis = 1)
#     print(tmp[:5])
print(df.shape)


(356255, 8)
NAME_TYPE_SUITE
(356255, 9)
NAME_INCOME_TYPE
(356255, 6)
NAME_EDUCATION_TYPE
(356255, 7)
NAME_FAMILY_STATUS
(356255, 7)
NAME_HOUSING_TYPE
(356255, 19)
OCCUPATION_TYPE
(356255, 5)
FONDKAPREMONT_MODE
(356255, 4)
HOUSETYPE_MODE
(356255, 8)
WALLSMATERIAL_MODE
(356255, 3)
EMERGENCYSTATE_MODE
(356255, 5)
FLAG_OTHER
(356255, 200)


# AREA

In [7]:
l = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG',
       'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
       'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG',
       'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE',
       'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE',
       'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE',
       'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI',
       'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI',
       'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
       'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
       'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE',
       'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']

In [8]:
df[l].fillna(0 , inplace = True)

/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [9]:
area = ['APARTMENTS','BASEMENTAREA', 'COMMONAREA','LANDAREA', 'LIVINGAPARTMENTS','LIVINGAREA',\
           'NONLIVINGAPARTMENTS', 'NONLIVINGAREA']

for x in ['_AVG', '_MODE', '_MEDI']:
    tmp = [y + x for y in area]
    print(tmp)
    df['livingsum' + x] = df[tmp].sum(axis=1)

['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'COMMONAREA_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']
['APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE']
['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI']


In [10]:
df.shape

(356255, 203)

# 1. Do the ratio wrt to count of family members & living area

In [11]:
# 1. Ratio of count of family members(for members less than 4) and living area

In [12]:
members_series = df.CNT_FAM_MEMBERS < 4

area_list = ['livingsum_AVG','livingsum_MODE','livingsum_MEDI']

In [13]:
df['members'] = np.nan

df['members'].loc[members_series] = df['CNT_FAM_MEMBERS'].loc[members_series]

for col in area_list:
    df[ratio_name(col, 'members')] = ratio(df, col, 'members')
    print(ratio_name(col, 'members'))
    
df.drop('members', inplace = True,axis = 1)

print(df.shape)

livingsum_AVG_divide_members
livingsum_MODE_divide_members
livingsum_MEDI_divide_members
(356255, 206)


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# 2. Multiply area rating & living area

In [14]:
# def multiply_name(col1, col2): return col1 + '_times_' + col2
# def multiply(df, col1, col2): return df[col1] * df[col2]

In [15]:
rating = ['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']
area_list = ['livingsum_AVG','livingsum_MODE','livingsum_MEDI']
for i in rating:
    for j in area_list:
        df[multiply_name(i, j)] = multiply(df, i, i)
        print(multiply_name(i, j))

print(df.shape)

REGION_RATING_CLIENT_times_livingsum_AVG
REGION_RATING_CLIENT_times_livingsum_MODE
REGION_RATING_CLIENT_times_livingsum_MEDI
REGION_RATING_CLIENT_W_CITY_times_livingsum_AVG
REGION_RATING_CLIENT_W_CITY_times_livingsum_MODE
REGION_RATING_CLIENT_W_CITY_times_livingsum_MEDI
(356255, 212)


# Do the ratio wrt to amt-dpd and region-population

In [16]:
obs = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
pop = ['REGION_POPULATION_RELATIVE']

In [17]:
for col in obs:
    df[ratio_name(col, 'REGION_POPULATION_RELATIVE')] = ratio(df, col, 'REGION_POPULATION_RELATIVE')
    print(ratio_name(col, 'REGION_POPULATION_RELATIVE'))
print(df.shape)

OBS_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
DEF_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
OBS_60_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
DEF_60_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
(356255, 216)


# DEF/OBS & 60/30

In [18]:
obs_30 = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE']
obs_60 = ['OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']

In [19]:
for i in obs_60:
    for j in obs_30:
        print(ratio_name(i,j))
        df[ratio_name(i,j)] = ratio(df, i, j)

OBS_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
OBS_60_CNT_SOCIAL_CIRCLE_divide_DEF_30_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE_divide_DEF_30_CNT_SOCIAL_CIRCLE


In [20]:
df.shape

(356255, 220)

In [21]:
default = ['DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
obs = ['OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE']

In [22]:
for i in default:
    for j in obs:
        print(ratio_name(i,j))
        df[ratio_name(i,j)] = ratio(df, i, j)
        print(df.shape)

DEF_30_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
(356255, 221)
DEF_30_CNT_SOCIAL_CIRCLE_divide_OBS_60_CNT_SOCIAL_CIRCLE
(356255, 222)
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
(356255, 222)
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_60_CNT_SOCIAL_CIRCLE
(356255, 223)


# Other Ratios:

In [23]:
can_be_numerator = [x for x in train.columns if 'AMT' in x] + ['OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']
can_be_denominator = [x for x in train.columns if 'CNT' in x or 'DAYS' in x] + can_be_numerator

print(len(can_be_numerator),can_be_numerator)
print(len(can_be_denominator),can_be_denominator)

12 ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']
23 ['CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']


In [24]:
i = 0
for x in can_be_numerator:
    for y in can_be_denominator:
        if y != x and ratio_name(y,x) not in df.columns:
            i += 1
            print(ratio_name(x,y))
            df[ratio_name(x, y)] = ratio(df, x, y)
print(i, df.shape)

AMT_INCOME_TOTAL_divide_CNT_CHILDREN
AMT_INCOME_TOTAL_divide_DAYS_BIRTH
AMT_INCOME_TOTAL_divide_DAYS_EMPLOYED
AMT_INCOME_TOTAL_divide_DAYS_REGISTRATION
AMT_INCOME_TOTAL_divide_DAYS_ID_PUBLISH
AMT_INCOME_TOTAL_divide_CNT_FAM_MEMBERS
AMT_INCOME_TOTAL_divide_OBS_30_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DEF_30_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_OBS_60_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DEF_60_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DAYS_LAST_PHONE_CHANGE
AMT_INCOME_TOTAL_divide_AMT_CREDIT
AMT_INCOME_TOTAL_divide_AMT_ANNUITY
AMT_INCOME_TOTAL_divide_AMT_GOODS_PRICE
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_HOUR
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_DAY
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_WEEK
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_MON
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_QRT
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_YEAR
AMT_INCOME_TOTAL_divide_OWN_CAR_AGE
AMT_INCOME_TOTAL_divide_DAYS_LAST_PHONE_CHANGE
AMT_CREDIT_divide_CNT_CHILDR

DAYS_LAST_PHONE_CHANGE_divide_DAYS_BIRTH
DAYS_LAST_PHONE_CHANGE_divide_DAYS_EMPLOYED
DAYS_LAST_PHONE_CHANGE_divide_DAYS_REGISTRATION
DAYS_LAST_PHONE_CHANGE_divide_DAYS_ID_PUBLISH
DAYS_LAST_PHONE_CHANGE_divide_CNT_FAM_MEMBERS
DAYS_LAST_PHONE_CHANGE_divide_OBS_30_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE_divide_DEF_30_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE_divide_OBS_60_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE_divide_DEF_60_CNT_SOCIAL_CIRCLE
197 (356255, 409)


# Concat prev and do features

In [25]:
prev = pd.read_pickle(PATH + 'test/prev2curr_0.pkl')
df = df.merge(prev, how = 'left', on = 'SK_ID_CURR')
print(df.shape)

(356255, 1226)


In [26]:
# Some ratios
amt_annuity = ['prev_AMT_ANNUITY_mean',
 'prev_AMT_ANNUITY_median']
#  'prev_AMT_ANNUITY_max',
#  'prev_AMT_ANNUITY_min',
#  'prev_AMT_ANNUITY_sum']
amt_credit = ['prev_AMT_CREDIT_mean','prev_AMT_CREDIT_median']
#  'prev_AMT_CREDIT_max',
#  'prev_AMT_CREDIT_min',
#  'prev_AMT_CREDIT_sum']
amt_goods_price = ['prev_AMT_GOODS_PRICE_mean','prev_AMT_GOODS_PRICE_median']
#  'prev_AMT_GOODS_PRICE_max',
#  'prev_AMT_GOODS_PRICE_min',
#  'prev_AMT_GOODS_PRICE_sum']
week = ['prev_WEEKDAY_APPR_PROCESS_START_mean',
 'prev_WEEKDAY_APPR_PROCESS_START_median']
#  'prev_WEEKDAY_APPR_PROCESS_START_max',
#  'prev_WEEKDAY_APPR_PROCESS_START_min',
#  'prev_WEEKDAY_APPR_PROCESS_START_sum']

list_df = ['AMT_ANNUITY','AMT_CREDIT', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START']
list_prev = [amt_annuity, amt_credit, amt_goods_price, week]
for i in range(4):
    for j in range(2):
        print(ratio_name(list_df[i], list_prev[i][j]))
        df[ratio_name(list_df[i], list_prev[i][j])] = ratio(df, list_df[i], list_prev[i][j])


AMT_ANNUITY_divide_prev_AMT_ANNUITY_mean
AMT_ANNUITY_divide_prev_AMT_ANNUITY_median
AMT_CREDIT_divide_prev_AMT_CREDIT_mean
AMT_CREDIT_divide_prev_AMT_CREDIT_median
AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_mean
AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_median
WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_mean
WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_median


In [27]:
# ratio about amt_income_total
# x = ['prev_AMT_APPLICATION_mean','prev_AMT_APPLICATION_max','prev_AMT_APPLICATION_min',\
#      'prev_AMT_ANNUITY_mean','prev_AMT_ANNUITY_max','prev_AMT_ANNUITY_min',\
#     'prev_AMT_CREDIT_mean','prev_AMT_CREDIT_max','prev_AMT_CREDIT_min']
x = ['prev_AMT_APPLICATION_mean','prev_AMT_ANNUITY_mean','prev_AMT_CREDIT_mean']
for i in x:
    print(ratio_name('AMT_INCOME_TOTAL',i))
    df[ratio_name('AMT_INCOME_TOTAL',i)] = ratio(df, 'AMT_INCOME_TOTAL', i)

AMT_INCOME_TOTAL_divide_prev_AMT_APPLICATION_mean
AMT_INCOME_TOTAL_divide_prev_AMT_ANNUITY_mean
AMT_INCOME_TOTAL_divide_prev_AMT_CREDIT_mean


In [28]:
# docs
# docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
# print(docs)
# df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1) 
# df.drop(docs,axis = 1, inplace = True)

In [29]:
# ext
# ext_col = [col for col in df.columns if 'EXT' in col]
# df['MEAN_EXT'] = df[ext_col].mean(axis = 1)

In [30]:
df.shape

(356255, 1237)

# Cat Features drop

In [31]:
# cat_feats = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE',
#     'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_EMAIL', 'OCCUPATION_TYPE',
#     'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
#     'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
#     'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
#     'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
#     'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
#     'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21','WEEKDAY_APPR_PROCESS_START', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 
#     'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'
# ]

# #'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'\n 'FLAG_PHONE'
# df.drop(cat_feats, inplace = True, axis = 1)
# print(df.shape)

df.drop(categorical_columns, axis = 1, inplace = True)
print(df.shape)

train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
print(train.shape,test.shape)

test.drop('TARGET', axis = 1, inplace = True)
test.shape

(356255, 1225)
(307511, 1225) (48744, 1225)


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(48744, 1224)

# Output

In [32]:
train.to_pickle(PATH + 'test/train_w_prev_2.pkl')
test.to_pickle(PATH + 'test/test_w_prev_2.pkl')

In [33]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_ANNUITY_divide_prev_AMT_ANNUITY_median,AMT_CREDIT_divide_prev_AMT_CREDIT_mean,AMT_CREDIT_divide_prev_AMT_CREDIT_median,AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_mean,AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_median,WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_mean,WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_median,AMT_INCOME_TOTAL_divide_prev_AMT_APPLICATION_mean,AMT_INCOME_TOTAL_divide_prev_AMT_ANNUITY_mean,AMT_INCOME_TOTAL_divide_prev_AMT_CREDIT_mean
0,100002,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,2.669812,2.270797,2.270797,1.960292,1.960292,NaN,NaN,1.130937,21.887692,1.130937
1,100003,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.552885,2.671472,3.710165,2.593949,3.346667,0.333333,0.250000,0.620067,4.774199,0.557631
2,100004,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,1.259975,6.714414,6.714414,5.559674,5.559674,0.250000,0.250000,2.779837,12.599748,3.357207
3,100006,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,1.114494,1.071948,1.167031,0.727398,0.752454,0.000000,0.000000,0.495953,5.707962,0.462811
4,100007,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,1.505441,3.078516,2.591793,3.407953,2.682353,0.666667,0.571429,0.807147,9.895100,0.729122


In [34]:
i =0
for col in train.columns:
    if (train[col].isnull().sum()/len(train))> 0.25:
        print(col, train[col].isnull().sum()/len(train))
        i+= 1
print(i)

OWN_CAR_AGE 0.6599081008484249
EXT_SOURCE_1 0.5638107254699832
APARTMENTS_AVG 0.5074972927797705
BASEMENTAREA_AVG 0.5851595552679416
YEARS_BEGINEXPLUATATION_AVG 0.48781019215572774
YEARS_BUILD_AVG 0.6649778381911542
COMMONAREA_AVG 0.6987229725115525
ELEVATORS_AVG 0.532959796560123
ENTRANCES_AVG 0.503487680115508
FLOORSMAX_AVG 0.4976082156410665
FLOORSMIN_AVG 0.6784862980511266
LANDAREA_AVG 0.5937673774271489
LIVINGAPARTMENTS_AVG 0.6835495315614726
LIVINGAREA_AVG 0.5019332641759157
NONLIVINGAPARTMENTS_AVG 0.6943296337366793
NONLIVINGAREA_AVG 0.5517916432257708
APARTMENTS_MODE 0.5074972927797705
BASEMENTAREA_MODE 0.5851595552679416
YEARS_BEGINEXPLUATATION_MODE 0.48781019215572774
YEARS_BUILD_MODE 0.6649778381911542
COMMONAREA_MODE 0.6987229725115525
ELEVATORS_MODE 0.532959796560123
ENTRANCES_MODE 0.503487680115508
FLOORSMAX_MODE 0.4976082156410665
FLOORSMIN_MODE 0.6784862980511266
LANDAREA_MODE 0.5937673774271489
LIVINGAPARTMENTS_MODE 0.6835495315614726
LIVINGAREA_MODE 0.5019332641759157